# Imports

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.mediation import Mediation

import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import pickle

# Set Directory Paths

In [2]:
# Location of the data folder
%store -r dataDir

# Directory path to load formatted gfap_ferritin_dataDir Data
%store -r gfap_ferritin_dataDir

# Directory path to save Calcualted Analysis Data
%store -r gfap_ferritin_CalData

# Loading GFAP, Ferritin Dataset - %AO

In [3]:
gfap_ferr_T = pd.read_csv(os.path.join(dataDir, 'GFAP_Ferritin', 'AO_DATASET_01_22_2024.csv'))

In [4]:
gfap_ferr_T

,AutopsyID,BlockID,Hemisphere,Region,Ferritin,GFAP,AT8,TDP43,INDDID,importID,0Cont1TDP2Tau,1ExcludeALSorLATE,1PSP0not,NPDx1
0,1986-003,1986-003-12F,N,MFC,NaN,0.054025,NaN,NaN,103121.0,103121.0,2,0,1,Progressive supranuclear palsy
1,1986-003,1986-003-12F,R,MFC,0.006197,NaN,NaN,NaN,103121.0,103121.0,2,0,1,Progressive supranuclear palsy
2,1986-003,1986-003-13F,R,M1,0.024898,0.054097,NaN,NaN,103121.0,103121.0,2,0,1,Progressive supranuclear palsy
3,1986-003,1986-003-61F,R,HIP,NaN,NaN,0.000154,NaN,103121.0,103121.0,2,0,1,Progressive supranuclear palsy
4,1995-005,1995-005-04E,R,OFC,0.026164,0.051511,NaN,0.004535,111517.0,111517.0,1,0,0,Frontotemporal lobar degeneration with TDP inc...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3449,2018-216,2018-216-27F,L,OFC,NaN,NaN,0.006361,NaN,119113.0,119113.0,2,0,0,Tauopathy unclassifiable
3450,2018-216,2018-216-28F,L,V1,NaN,NaN,0.000256,NaN,119113.0,119113.0,2,0,0,Tauopathy unclassifiable
3451,2018-216,2018-216-29F,L,ANG,NaN,NaN,0.007745,NaN,119113.0,119113.0,2,0,0,Tauopathy unclassifiable
3452,2018-216,2018-216-30F,L,aCING,NaN,NaN,0.004663,NaN,119113.0,119113.0,2,0,0,Tauopathy unclassifiable


# Load IronGlia_FTLD_INDDIDs Data

In [5]:
# ironGlia_T = pd.read_excel(os.path.join(dataDir, 'GFAP_Ferritin', 'Copy of IronGlia_FTLD_INDDIDs Neuopath dx code.xlsx'))

# Merging DataFrames by INDDID

In [6]:
# gfap_ferr_T_merged = pd.merge(gfap_ferr_T, ironGlia_T, on='INDDID', how='outer')[['INDDID', 'AutopsyID', 'BlockID', 'Hemisphere', 'Region', 'Ferritin', 'GFAP', 'AT8', 'TDP43', '0Cont1TDP2Tau', '1ExcludeALSorLATE']]

In [7]:
# gfap_ferr_T_merged 

## Exclude where 1ExcludeALSorLATE == 1

In [8]:
gfap_ferr_T_merged = gfap_ferr_T
gfap_ferr_T_merged = gfap_ferr_T_merged[gfap_ferr_T_merged['1ExcludeALSorLATE'] != 1]

## Format the GFAP, Ferritin Data - %AO to desired format

### Get Tau(AT8) vs TDP43 Groups

In [9]:
gfap_ferr_type = gfap_ferr_T_merged.groupby('0Cont1TDP2Tau')

# HC
hc_T = gfap_ferr_type.get_group(0)

# Tau
tau_T = gfap_ferr_type.get_group(2)

# TDP
tdp_T = gfap_ferr_type.get_group(1)


# AutopsyID
hc_INDDIDs = hc_T['INDDID'].unique().tolist()
tau_INDDIDs = tau_T['INDDID'].unique().tolist()
tdp_INDDIDs = tdp_T['INDDID'].unique().tolist()
print(f'Number of Unique HC INDDIDs is: {len(hc_INDDIDs)}')
print(f'Number of Unique TAU INDDIDs is: {len(tau_INDDIDs)}')
print(f'Number of Unique TDP INDDIDs is: {len(tdp_INDDIDs)}')

# Get rid of rows where AT8 is NaN for TAU
tau_T = tau_T[tau_T['AT8'].notna()]

# Get rid of rows where TDP43 is NaN for TDP43
tdp_T = tdp_T[tdp_T['TDP43'].notna()]

Number of Unique HC INDDIDs is: 34
Number of Unique TAU INDDIDs is: 141
Number of Unique TDP INDDIDs is: 99


In [10]:
tau_T

,AutopsyID,BlockID,Hemisphere,Region,Ferritin,GFAP,AT8,TDP43,INDDID,importID,0Cont1TDP2Tau,1ExcludeALSorLATE,1PSP0not,NPDx1
3,1986-003,1986-003-61F,R,HIP,NaN,NaN,0.000154,NaN,103121.0,103121.0,2,0,1,Progressive supranuclear palsy
9,1995-057,1995-057-03E,L,MFC,0.033260,0.040396,0.133789,NaN,110745.0,110745.0,2,0,0,Corticobasal degeneration
10,1995-057,1995-057-04E,L,OFC,0.025673,0.017537,0.126244,NaN,110745.0,110745.0,2,0,0,Corticobasal degeneration
13,1995-057,1995-057-05E,L,pSTC,NaN,NaN,0.112106,NaN,110745.0,110745.0,2,0,0,Corticobasal degeneration
14,1995-057,1995-057-18E,L,HIP,NaN,NaN,0.057006,NaN,110745.0,110745.0,2,0,0,Corticobasal degeneration
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3449,2018-216,2018-216-27F,L,OFC,NaN,NaN,0.006361,NaN,119113.0,119113.0,2,0,0,Tauopathy unclassifiable
3450,2018-216,2018-216-28F,L,V1,NaN,NaN,0.000256,NaN,119113.0,119113.0,2,0,0,Tauopathy unclassifiable
3451,2018-216,2018-216-29F,L,ANG,NaN,NaN,0.007745,NaN,119113.0,119113.0,2,0,0,Tauopathy unclassifiable
3452,2018-216,2018-216-30F,L,aCING,NaN,NaN,0.004663,NaN,119113.0,119113.0,2,0,0,Tauopathy unclassifiable


In [11]:
tdp_T

,AutopsyID,BlockID,Hemisphere,Region,Ferritin,GFAP,AT8,TDP43,INDDID,importID,0Cont1TDP2Tau,1ExcludeALSorLATE,1PSP0not,NPDx1
4,1995-005,1995-005-04E,R,OFC,0.026164,0.051511,NaN,0.004535,111517.0,111517.0,1,0,0,Frontotemporal lobar degeneration with TDP inc...
5,1995-005,1995-005-08E,R,ANG,0.022526,NaN,NaN,0.002226,111517.0,111517.0,1,0,0,Frontotemporal lobar degeneration with TDP inc...
6,1995-005,1995-005-02E,R,M1,NaN,NaN,NaN,0.000940,111517.0,111517.0,1,0,0,Frontotemporal lobar degeneration with TDP inc...
7,1995-005,1995-005-06E,R,pSTC,NaN,NaN,NaN,0.000495,111517.0,111517.0,1,0,0,Frontotemporal lobar degeneration with TDP inc...
8,1995-005,1995-005-17E,R,HIP,NaN,NaN,NaN,0.003235,111517.0,111517.0,1,0,0,Frontotemporal lobar degeneration with TDP inc...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3393,2016-126,2016-126-28F,L,MFC,NaN,NaN,NaN,0.000013,120720.0,120720.0,1,0,0,Amyotrophic lateral sclerosis
3394,2016-126,2016-126-30F,L,OFC,NaN,NaN,NaN,0.000222,120720.0,120720.0,1,0,0,Amyotrophic lateral sclerosis
3395,2016-126,2016-126-31F,L,V1,NaN,NaN,NaN,0.000128,120720.0,120720.0,1,0,0,Amyotrophic lateral sclerosis
3396,2016-126,2016-126-33F,L,aCING,NaN,NaN,NaN,0.000065,120720.0,120720.0,1,0,0,Amyotrophic lateral sclerosis


#### Common INDDIDs between TAU vs TDP43

In [12]:
# Convert lists to sets and find the intersection
common_elements = set(tau_INDDIDs).intersection(tdp_INDDIDs)

# Check if there are any common elements
if common_elements:
    print("Common elements:", common_elements)
else:
    print("No common elements")

No common elements


## TAU - MFC, ANG, M1, VIS

### MFC: TAU, Ferritin %AO

In [13]:
# Get MFC Region - TAU
tau_region_type = tau_T.groupby('Region')
tau_MFC = tau_region_type.get_group('MFC')

# Remove rows where Ferritin Value is NaN
tau_MFC = tau_MFC.dropna(subset=['Ferritin'])

# Count non NaN and non zero values for each column
non_zero_non_nan_counts = (tau_MFC != 0) & tau_MFC.notna()
counts = non_zero_non_nan_counts.sum()

print(counts)

# Rename Columns / Drop TDP43
tau_MFC = tau_MFC.drop(columns=['Region', 'TDP43'])
tau_MFC = tau_MFC.rename(columns=lambda x: x + '_MFC')

AutopsyID            80
BlockID              80
Hemisphere           80
Region               80
Ferritin             80
GFAP                 60
AT8                  80
TDP43                 0
INDDID               80
importID             80
0Cont1TDP2Tau        80
1ExcludeALSorLATE     0
1PSP0not             34
NPDx1                80
dtype: int64


### OFC: TAU, Ferritin %AO

In [14]:
# Get MFC Region - TAU
tau_region_type = tau_T.groupby('Region')
tau_OFC = tau_region_type.get_group('OFC')

# Remove rows where Ferritin Value is NaN
tau_OFC = tau_OFC.dropna(subset=['Ferritin'])

# Count non NaN and non zero values for each column
non_zero_non_nan_counts = (tau_OFC != 0) & tau_OFC.notna()
counts = non_zero_non_nan_counts.sum()

print(counts)

# Rename Columns / Drop TDP43
tau_OFC = tau_OFC.drop(columns=['Region', 'TDP43'])
tau_OFC = tau_OFC.rename(columns=lambda x: x + '_OFC')

AutopsyID            72
BlockID              72
Hemisphere           72
Region               72
Ferritin             72
GFAP                 53
AT8                  72
TDP43                 0
INDDID               72
importID             72
0Cont1TDP2Tau        72
1ExcludeALSorLATE     0
1PSP0not             32
NPDx1                72
dtype: int64


### ANG: TAU, Ferritin %AO

In [15]:
# Get MFC Region - TAU
tau_region_type = tau_T.groupby('Region')
tau_ANG = tau_region_type.get_group('ANG')

# Remove rows where Ferritin Value is NaN
tau_ANG = tau_ANG.dropna(subset=['Ferritin'])

# Count non NaN and non zero values for each column
non_zero_non_nan_counts = (tau_ANG != 0) & tau_ANG.notna()
counts = non_zero_non_nan_counts.sum()

print(counts)

# Rename Columns / Drop TDP43
tau_ANG = tau_ANG.drop(columns=['Region', 'TDP43'])
tau_ANG = tau_ANG.rename(columns=lambda x: x + '_ANG')

AutopsyID            54
BlockID              54
Hemisphere           54
Region               54
Ferritin             54
GFAP                 22
AT8                  54
TDP43                 0
INDDID               54
importID             54
0Cont1TDP2Tau        54
1ExcludeALSorLATE     0
1PSP0not             16
NPDx1                54
dtype: int64


### M1: TAU, Ferritin %AO

In [16]:
# Get M1 Region - TAU
tau_region_type = tau_T.groupby('Region')
tau_M1 = tau_region_type.get_group('M1')

# Remove rows where Ferritin Value is NaN
tau_M1 = tau_M1.dropna(subset=['Ferritin'])

# Count non NaN and non zero values for each column
non_zero_non_nan_counts = (tau_M1 != 0) & tau_M1.notna()
counts = non_zero_non_nan_counts.sum()

print(counts)

# Rename Columns / Drop TDP43
tau_M1 = tau_M1.drop(columns=['Region', 'TDP43'])
tau_M1 = tau_M1.rename(columns=lambda x: x + '_M1')

AutopsyID            15
BlockID              15
Hemisphere           15
Region               15
Ferritin             15
GFAP                 14
AT8                  15
TDP43                 0
INDDID               15
importID             15
0Cont1TDP2Tau        15
1ExcludeALSorLATE     0
1PSP0not             12
NPDx1                15
dtype: int64


### V1: TAU, Ferritin %AO

In [17]:
# Get V1 Region - TAU
tau_region_type = tau_T.groupby('Region')
tau_V1 = tau_region_type.get_group('V1')

# Remove rows where Ferritin Value is NaN
tau_V1 = tau_V1.dropna(subset=['Ferritin'])

# Count non NaN and non zero values for each column
non_zero_non_nan_counts = (tau_V1 != 0) & tau_V1.notna()
counts = non_zero_non_nan_counts.sum()

print(counts)

# Rename Columns / Drop TDP43
tau_V1 = tau_V1.drop(columns=['Region', 'TDP43'])
tau_V1 = tau_V1.rename(columns=lambda x: x + '_V1')

AutopsyID            21
BlockID              21
Hemisphere           21
Region               21
Ferritin             21
GFAP                 12
AT8                  21
TDP43                 0
INDDID               21
importID             21
0Cont1TDP2Tau        21
1ExcludeALSorLATE     0
1PSP0not              9
NPDx1                21
dtype: int64


### Merging

In [18]:
# Merge MFC, ANG based on AutopsyID & Hemisphere
TAU_MFC_ANG = pd.merge(tau_MFC, tau_ANG, left_on=['AutopsyID_MFC', 'Hemisphere_MFC'],  
                                         right_on=['AutopsyID_ANG', 'Hemisphere_ANG'], how='inner')

TAU_MFC_ANG = TAU_MFC_ANG.drop(columns=['AutopsyID_ANG', 'Hemisphere_ANG'])
TAU_MFC_ANG = TAU_MFC_ANG.rename(columns={'AutopsyID_MFC': 'AutopsyID',
                                          'Hemisphere_MFC': 'Hemisphere'})

In [19]:
TAU_MFC_ANG

,AutopsyID,BlockID_MFC,Hemisphere,Ferritin_MFC,GFAP_MFC,AT8_MFC,INDDID_MFC,importID_MFC,0Cont1TDP2Tau_MFC,1ExcludeALSorLATE_MFC,...,BlockID_ANG,Ferritin_ANG,GFAP_ANG,AT8_ANG,INDDID_ANG,importID_ANG,0Cont1TDP2Tau_ANG,1ExcludeALSorLATE_ANG,1PSP0not_ANG,NPDx1_ANG
0,2005-073,2005-073-29F,R,0.015930,0.034787,0.098714,102149.00,102149.00,2,0,...,2005-073-34F,0.008880,NaN,0.070316,102149.00,102149.00,2,0,0,Corticobasal degeneration
1,2006-027,2006-027-04F,L,0.010866,0.022061,0.010569,113113.00,113113.00,2,0,...,2006-027-09F,0.017863,NaN,0.006209,113113.00,113113.00,2,0,0,Tauopathy unclassifiable
2,2008-251,2008-251-29F,L,0.028530,0.045559,0.254047,107516.00,107516.00,2,0,...,2008-251-32F,0.027772,NaN,0.310358,107516.00,107516.00,2,0,0,Corticobasal degeneration
3,2009-102,2009-102-25F,R,0.006274,0.028020,0.000332,115327.00,115327.00,2,0,...,2009-102-28F,0.018715,NaN,0.000395,115327.00,115327.00,2,0,1,Progressive supranuclear palsy
4,2011-203,2011-203-25F,L,0.008894,0.037125,0.000304,100518.00,100518.00,2,0,...,2011-203-28F,0.017492,0.052037,0.000200,100518.00,100518.00,2,0,0,Tauopathy unclassifiable
5,2013-208,2013-208-25F,R,0.019375,0.049065,0.011498,116831.00,116831.00,2,0,...,2013-208-28F,0.009295,0.056224,0.008309,116831.00,116831.00,2,0,0,Corticobasal degeneration
6,2013-220,2013-220-25F,R,0.021653,0.041816,0.000725,116283.00,116283.00,2,0,...,2013-220-29F,0.015637,0.035068,0.001012,116283.00,116283.00,2,0,1,Progressive supranuclear palsy
7,2013-223,2013-223-06F,L,0.020343,0.063952,0.125530,108026.00,108026.00,2,0,...,2013-223-03F,0.024613,NaN,0.046466,108026.00,108026.00,2,0,0,Pick's disease
8,2014-011,2014-011-28F,L,0.026754,NaN,0.004994,111231.00,111231.00,2,0,...,2014-011-32F,0.027113,NaN,0.002966,111231.00,111231.00,2,0,1,Progressive supranuclear palsy
9,2014-052,2014-052-26F,R,0.017498,0.036848,0.007585,116607.00,116607.00,2,0,...,2014-052-30F,0.017715,NaN,0.022633,116607.00,116607.00,2,0,1,Progressive supranuclear palsy


In [20]:
# Merge MFC, ANG based on AutopsyID & Hemisphere
TAU_OFC_ANG = pd.merge(tau_OFC, tau_ANG, left_on=['AutopsyID_OFC', 'Hemisphere_OFC'],  
                                         right_on=['AutopsyID_ANG', 'Hemisphere_ANG'], how='inner')

TAU_OFC_ANG = TAU_OFC_ANG.drop(columns=['AutopsyID_ANG', 'Hemisphere_ANG'])
TAU_OFC_ANG = TAU_OFC_ANG.rename(columns={'AutopsyID_OFC': 'AutopsyID',
                                          'Hemisphere_OFC': 'Hemisphere'})

In [21]:
TAU_OFC_ANG

,AutopsyID,BlockID_OFC,Hemisphere,Ferritin_OFC,GFAP_OFC,AT8_OFC,INDDID_OFC,importID_OFC,0Cont1TDP2Tau_OFC,1ExcludeALSorLATE_OFC,...,BlockID_ANG,Ferritin_ANG,GFAP_ANG,AT8_ANG,INDDID_ANG,importID_ANG,0Cont1TDP2Tau_ANG,1ExcludeALSorLATE_ANG,1PSP0not_ANG,NPDx1_ANG
0,2007-003,2007-003-25F,R,0.011096,0.034525,0.065233,114348.00,114348.00,2,0,...,2007-003-27F,0.014662,NaN,0.238042,114348.00,114348.00,2,0,1,Progressive supranuclear palsy
1,2008-251,2008-251-55F,L,0.025272,0.055223,0.136122,107516.00,107516.00,2,0,...,2008-251-32F,0.027772,NaN,0.310358,107516.00,107516.00,2,0,0,Corticobasal degeneration
2,2009-102,2009-102-26F,R,0.008658,0.033446,0.003335,115327.00,115327.00,2,0,...,2009-102-28F,0.018715,NaN,0.000395,115327.00,115327.00,2,0,1,Progressive supranuclear palsy
3,2010-015,2010-015-32F,R,0.025353,0.075226,0.027926,107187.00,107187.00,2,0,...,2010-015-34F,0.021949,NaN,0.011312,107187.00,107187.00,2,0,0,Pick's disease
4,2011-203,2011-203-26F,L,0.014962,0.030565,0.000059,100518.00,100518.00,2,0,...,2011-203-28F,0.017492,0.052037,0.000200,100518.00,100518.00,2,0,0,Tauopathy unclassifiable
5,2013-220,2013-220-27F,R,0.011491,0.042502,0.002642,116283.00,116283.00,2,0,...,2013-220-29F,0.015637,0.035068,0.001012,116283.00,116283.00,2,0,1,Progressive supranuclear palsy
6,2013-223,2013-223-07F,L,0.014895,0.107302,0.007129,108026.00,108026.00,2,0,...,2013-223-03F,0.024613,NaN,0.046466,108026.00,108026.00,2,0,0,Pick's disease
7,2014-011,2014-011-30F,L,0.018053,NaN,0.001812,111231.00,111231.00,2,0,...,2014-011-32F,0.027113,NaN,0.002966,111231.00,111231.00,2,0,1,Progressive supranuclear palsy
8,2014-212,2014-212-26F,L,0.015978,0.031675,0.003125,116591.00,116591.00,2,0,...,2014-212-28F,0.020828,NaN,0.007330,116591.00,116591.00,2,0,1,Progressive supranuclear palsy
9,2014-251,2014-251-25F,L,0.019426,NaN,0.037876,116504.00,116504.00,2,0,...,2014-251-27F,0.020757,NaN,0.036539,116504.00,116504.00,2,0,0,Corticobasal degeneration


In [22]:
# Merge M1, V1 based on AutopsyID & Hemisphere
TAU_M1_V1 = pd.merge(tau_M1, tau_V1, left_on=['AutopsyID_M1', 'Hemisphere_M1'],  
                                         right_on=['AutopsyID_V1', 'Hemisphere_V1'], how='inner')

TAU_M1_V1 = TAU_M1_V1.drop(columns=['AutopsyID_V1', 'Hemisphere_V1'])
TAU_M1_V1 = TAU_M1_V1.rename(columns={'AutopsyID_M1': 'AutopsyID',
                                          'Hemisphere_M1': 'Hemisphere'})

In [23]:
TAU_M1_V1

,AutopsyID,BlockID_M1,Hemisphere,Ferritin_M1,GFAP_M1,AT8_M1,INDDID_M1,importID_M1,0Cont1TDP2Tau_M1,1ExcludeALSorLATE_M1,...,BlockID_V1,Ferritin_V1,GFAP_V1,AT8_V1,INDDID_V1,importID_V1,0Cont1TDP2Tau_V1,1ExcludeALSorLATE_V1,1PSP0not_V1,NPDx1_V1
0,2000-151,2000-151-30F,L,0.015007,0.035296,0.024328,113938.0,113938.0,2,0,...,2000-151-36F,0.011267,0.043683,0.000040,113938.0,113938.0,2,0,1,Progressive supranuclear palsy
1,2003-105,2003-105-01F,L,0.016095,0.052224,0.038664,109115.0,109115.0,2,0,...,2003-105-14F,0.002551,0.051165,0.000237,109115.0,109115.0,2,0,0,Corticobasal degeneration
2,2007-102,2007-102-01F,L,0.024682,0.041001,0.010258,109299.0,109299.0,2,0,...,2007-102-07F,0.013451,NaN,0.000064,109299.0,109299.0,2,0,1,Progressive supranuclear palsy
3,2008-166,2008-166-25F,R,0.012269,0.038704,0.119794,105564.0,105564.0,2,0,...,2008-166-29F,0.014030,0.047593,0.000093,105564.0,105564.0,2,0,0,Pick's disease
4,2009-042,2009-042-26F,L,0.011550,0.040270,0.002439,112570.0,112570.0,2,0,...,2009-042-30F,0.018264,NaN,0.000225,112570.0,112570.0,2,0,1,Progressive supranuclear palsy
5,2009-073,2009-073-31F,L,0.016434,0.036770,0.000616,108077.0,108077.0,2,0,...,2009-073-35F,0.017721,NaN,0.000112,108077.0,108077.0,2,0,1,Progressive supranuclear palsy
6,2013-216,2013-216-23F,R,0.013827,0.052921,0.001686,118575.0,118575.0,2,0,...,2013-216-28F,0.014301,NaN,0.000796,118575.0,118575.0,2,0,1,Progressive supranuclear palsy
7,2015-194,2015-194-23F,L,0.010735,0.071366,0.170649,108508.0,108508.0,2,0,...,2015-194-27F,0.010321,NaN,0.000121,108508.0,108508.0,2,0,0,Pick's disease
8,2015-253,2015-253-23F,L,0.019449,0.039955,0.011290,120298.0,120298.0,2,0,...,2015-253-28F,0.012675,NaN,0.000214,120298.0,120298.0,2,0,1,Progressive supranuclear palsy
9,2016-124,2016-124-23F,L,0.023962,0.047645,0.047391,106840.0,106840.0,2,0,...,2016-124-28F,0.012561,0.054559,0.000238,106840.0,106840.0,2,0,1,Progressive supranuclear palsy


In [24]:
TAU_M1_V1_PSP1 = TAU_M1_V1[TAU_M1_V1['1PSP0not_V1'] == 1]

In [25]:
TAU_M1_V1_PSP1

,AutopsyID,BlockID_M1,Hemisphere,Ferritin_M1,GFAP_M1,AT8_M1,INDDID_M1,importID_M1,0Cont1TDP2Tau_M1,1ExcludeALSorLATE_M1,...,BlockID_V1,Ferritin_V1,GFAP_V1,AT8_V1,INDDID_V1,importID_V1,0Cont1TDP2Tau_V1,1ExcludeALSorLATE_V1,1PSP0not_V1,NPDx1_V1
0,2000-151,2000-151-30F,L,0.015007,0.035296,0.024328,113938.0,113938.0,2,0,...,2000-151-36F,0.011267,0.043683,0.000040,113938.0,113938.0,2,0,1,Progressive supranuclear palsy
2,2007-102,2007-102-01F,L,0.024682,0.041001,0.010258,109299.0,109299.0,2,0,...,2007-102-07F,0.013451,NaN,0.000064,109299.0,109299.0,2,0,1,Progressive supranuclear palsy
4,2009-042,2009-042-26F,L,0.011550,0.040270,0.002439,112570.0,112570.0,2,0,...,2009-042-30F,0.018264,NaN,0.000225,112570.0,112570.0,2,0,1,Progressive supranuclear palsy
5,2009-073,2009-073-31F,L,0.016434,0.036770,0.000616,108077.0,108077.0,2,0,...,2009-073-35F,0.017721,NaN,0.000112,108077.0,108077.0,2,0,1,Progressive supranuclear palsy
6,2013-216,2013-216-23F,R,0.013827,0.052921,0.001686,118575.0,118575.0,2,0,...,2013-216-28F,0.014301,NaN,0.000796,118575.0,118575.0,2,0,1,Progressive supranuclear palsy
8,2015-253,2015-253-23F,L,0.019449,0.039955,0.011290,120298.0,120298.0,2,0,...,2015-253-28F,0.012675,NaN,0.000214,120298.0,120298.0,2,0,1,Progressive supranuclear palsy
9,2016-124,2016-124-23F,L,0.023962,0.047645,0.047391,106840.0,106840.0,2,0,...,2016-124-28F,0.012561,0.054559,0.000238,106840.0,106840.0,2,0,1,Progressive supranuclear palsy
10,2017-171,2017-171-23F,R,0.010573,0.018218,0.011328,104937.0,104937.0,2,0,...,2017-171-27F,0.007066,0.058472,0.000105,104937.0,104937.0,2,0,1,Progressive supranuclear palsy


In [26]:
# Merge MFC, V1 based on AutopsyID & Hemisphere
TAU_MFC_V1 = pd.merge(tau_MFC, tau_V1, left_on=['AutopsyID_MFC', 'Hemisphere_MFC'],  
                                         right_on=['AutopsyID_V1', 'Hemisphere_V1'], how='inner')

TAU_MFC_V1 = TAU_MFC_V1.drop(columns=['AutopsyID_V1', 'Hemisphere_V1'])
TAU_MFC_V1 = TAU_MFC_V1.rename(columns={'AutopsyID_MFC': 'AutopsyID',
                                          'Hemisphere_MFC': 'Hemisphere'})

In [27]:
TAU_MFC_V1_PSP1 = TAU_MFC_V1[TAU_MFC_V1['1PSP0not_V1'] == 1]

In [28]:
# Merge MFC, V1 based on AutopsyID & Hemisphere
TAU_OFC_V1 = pd.merge(tau_OFC, tau_V1, left_on=['AutopsyID_OFC', 'Hemisphere_OFC'],  
                                         right_on=['AutopsyID_V1', 'Hemisphere_V1'], how='inner')

TAU_OFC_V1 = TAU_OFC_V1.drop(columns=['AutopsyID_V1', 'Hemisphere_V1'])
TAU_OFC_V1 = TAU_OFC_V1.rename(columns={'AutopsyID_OFC': 'AutopsyID',
                                          'Hemisphere_OFC': 'Hemisphere'})

In [29]:
TAU_OFC_V1_PSP1 = TAU_OFC_V1[TAU_OFC_V1['1PSP0not_V1'] == 1]

## TDP - MFC, ANG

### MFC: TDP, Ferritin %AO

In [30]:
# Get MFC Region - TAU
tdp_region_type = tdp_T.groupby('Region')
tdp_MFC = tdp_region_type.get_group('MFC')

# Remove rows where Ferritin Value is NaN
tdp_MFC = tdp_MFC.dropna(subset=['Ferritin'])

# Count non NaN and non zero values for each column
non_zero_non_nan_counts = (tdp_MFC != 0) & tdp_MFC.notna()
counts = non_zero_non_nan_counts.sum()

print(counts)

# Rename Columns / Drop TDP43
tdp_MFC = tdp_MFC.drop(columns=['Region', 'AT8'])
tdp_MFC = tdp_MFC.rename(columns=lambda x: x + '_MFC')

AutopsyID            64
BlockID              64
Hemisphere           64
Region               64
Ferritin             64
GFAP                 61
AT8                   0
TDP43                64
INDDID               64
importID             64
0Cont1TDP2Tau        64
1ExcludeALSorLATE     0
1PSP0not              0
NPDx1                64
dtype: int64


### OFC: TDP, Ferritin %AO

In [31]:
# Get MFC Region - TAU
tdp_region_type = tdp_T.groupby('Region')
tdp_OFC = tdp_region_type.get_group('OFC')

# Remove rows where Ferritin Value is NaN
tdp_OFC = tdp_OFC.dropna(subset=['Ferritin'])

# Count non NaN and non zero values for each column
non_zero_non_nan_counts = (tdp_OFC != 0) & tdp_OFC.notna()
counts = non_zero_non_nan_counts.sum()

print(counts)

# Rename Columns / Drop TDP43
tdp_OFC = tdp_OFC.drop(columns=['Region', 'AT8'])
tdp_OFC = tdp_OFC.rename(columns=lambda x: x + '_OFC')

AutopsyID            90
BlockID              90
Hemisphere           90
Region               90
Ferritin             90
GFAP                 84
AT8                   0
TDP43                90
INDDID               90
importID             90
0Cont1TDP2Tau        90
1ExcludeALSorLATE     0
1PSP0not              0
NPDx1                90
dtype: int64


### ANG: TDP, Ferritin %AO

In [32]:
# Get MFC Region - TAU
tdp_region_type = tdp_T.groupby('Region')
tdp_ANG = tdp_region_type.get_group('ANG')

# Remove rows where Ferritin Value is NaN
tdp_ANG = tdp_ANG.dropna(subset=['Ferritin'])

# Count non NaN and non zero values for each column
non_zero_non_nan_counts = (tdp_ANG != 0) & tdp_ANG.notna()
counts = non_zero_non_nan_counts.sum()

print(counts)

# Rename Columns / Drop TDP43
tdp_ANG = tdp_ANG.drop(columns=['Region', 'AT8'])
tdp_ANG = tdp_ANG.rename(columns=lambda x: x + '_ANG')

AutopsyID            51
BlockID              51
Hemisphere           51
Region               51
Ferritin             51
GFAP                 12
AT8                   0
TDP43                51
INDDID               51
importID             51
0Cont1TDP2Tau        51
1ExcludeALSorLATE     0
1PSP0not              0
NPDx1                51
dtype: int64


### M1: TDP, Ferritin %AO

In [33]:
# Get MFC Region - TAU
tdp_region_type = tdp_T.groupby('Region')
tdp_M1 = tdp_region_type.get_group('M1')

# Remove rows where Ferritin Value is NaN
tdp_M1 = tdp_M1.dropna(subset=['Ferritin'])

# Count non NaN and non zero values for each column
non_zero_non_nan_counts = (tdp_M1 != 0) & tdp_M1.notna()
counts = non_zero_non_nan_counts.sum()

print(counts)

# Rename Columns / Drop TDP43
tdp_M1 = tdp_M1.drop(columns=['Region', 'AT8'])
tdp_M1 = tdp_M1.rename(columns=lambda x: x + '_M1')

AutopsyID            17
BlockID              17
Hemisphere           17
Region               17
Ferritin             17
GFAP                 16
AT8                   0
TDP43                17
INDDID               17
importID             17
0Cont1TDP2Tau        17
1ExcludeALSorLATE     0
1PSP0not              0
NPDx1                17
dtype: int64


### V1: TDP, Ferritin %AO

In [34]:
# Get MFC Region - TAU
tdp_region_type = tdp_T.groupby('Region')
tdp_V1 = tdp_region_type.get_group('V1')

# Remove rows where Ferritin Value is NaN
tdp_V1 = tdp_V1.dropna(subset=['Ferritin'])

# Count non NaN and non zero values for each column
non_zero_non_nan_counts = (tdp_V1 != 0) & tdp_V1.notna()
counts = non_zero_non_nan_counts.sum()

print(counts)

# Rename Columns / Drop TDP43
tdp_V1 = tdp_V1.drop(columns=['Region', 'AT8'])
tdp_V1 = tdp_V1.rename(columns=lambda x: x + '_V1')

AutopsyID            17
BlockID              17
Hemisphere           17
Region               17
Ferritin             17
GFAP                 14
AT8                   0
TDP43                17
INDDID               17
importID             17
0Cont1TDP2Tau        17
1ExcludeALSorLATE     0
1PSP0not              0
NPDx1                17
dtype: int64


### Merging

In [35]:
# Merge MFC, ANG based on AutopsyID & Hemisphere
TDP_MFC_ANG = pd.merge(tdp_MFC, tdp_ANG, left_on=['AutopsyID_MFC', 'Hemisphere_MFC'],  
                                         right_on=['AutopsyID_ANG', 'Hemisphere_ANG'], how='inner')

TDP_MFC_ANG = TDP_MFC_ANG.drop(columns=['AutopsyID_ANG', 'Hemisphere_ANG'])
TDP_MFC_ANG = TDP_MFC_ANG.rename(columns={'AutopsyID_MFC': 'AutopsyID',
                                          'Hemisphere_MFC': 'Hemisphere'})

In [36]:
TDP_MFC_ANG

,AutopsyID,BlockID_MFC,Hemisphere,Ferritin_MFC,GFAP_MFC,TDP43_MFC,INDDID_MFC,importID_MFC,0Cont1TDP2Tau_MFC,1ExcludeALSorLATE_MFC,...,BlockID_ANG,Ferritin_ANG,GFAP_ANG,TDP43_ANG,INDDID_ANG,importID_ANG,0Cont1TDP2Tau_ANG,1ExcludeALSorLATE_ANG,1PSP0not_ANG,NPDx1_ANG
0,1999-224,1999-224-01E,L,0.042623,0.042827,0.002274,112298.0,112298.0,1,0,...,1999-224-35F,0.009827,NaN,0.001056,112298.0,112298.0,1,0,0,Frontotemporal lobar degeneration with TDP inc...
1,2002-238,2002-238-15F,L,0.024325,0.034106,0.000813,110658.0,110658.0,1,0,...,2002-238-18F,0.017602,NaN,0.004022,110658.0,110658.0,1,0,0,Frontotemporal lobar degeneration with TDP inc...
2,2004-008,2004-008-04F,L,0.027765,0.030679,0.000526,110338.0,110338.0,1,0,...,2004-008-13F,0.021446,NaN,0.000265,110338.0,110338.0,1,0,0,Frontotemporal lobar degeneration with TDP inc...
3,2004-158,2004-158-26F,L,0.019065,0.030841,0.001851,114076.0,114076.0,1,0,...,2004-158-31F,0.015413,NaN,0.001289,114076.0,114076.0,1,0,0,Frontotemporal lobar degeneration with TDP inc...
4,2004-262,2004-262-26F,L,0.019302,0.022704,0.000424,110361.0,110361.0,1,0,...,2004-262-31F,0.022496,NaN,0.000579,110361.0,110361.0,1,0,0,Frontotemporal lobar degeneration with TDP inc...
5,2005-208,2005-208-24F,L,0.009474,0.046669,0.000646,104659.0,104659.0,1,0,...,2005-208-27F,0.013243,NaN,0.000379,104659.0,104659.0,1,0,0,Frontotemporal lobar degeneration with TDP inc...
6,2007-111,2007-111-03E,R,0.019225,0.039983,0.003048,104094.0,104094.0,1,0,...,2007-111-27F,0.013677,NaN,0.002862,104094.0,104094.0,1,0,0,Frontotemporal lobar degeneration with TDP inc...
7,2008-001,2008-001-23F,R,0.015520,0.026493,0.000076,111092.0,111092.0,1,0,...,2008-001-26F,0.020279,NaN,0.000002,111092.0,111092.0,1,0,0,Frontotemporal lobar degeneration with TDP inc...
8,2008-156,2008-156-32F,R,0.012718,0.018712,0.000031,101045.0,101045.0,1,0,...,2008-156-07E,0.007257,NaN,0.000130,101045.0,101045.0,1,0,0,Amyotrophic lateral sclerosis
9,2008-205,2008-205-30F,L,0.023678,0.053424,0.000008,105686.0,105686.0,1,0,...,2008-205-34F,0.019702,NaN,0.000010,105686.0,105686.0,1,0,0,Frontotemporal lobar degeneration with TDP inc...


In [37]:
# Merge MFC, ANG based on AutopsyID & Hemisphere
TDP_OFC_ANG = pd.merge(tdp_OFC, tdp_ANG, left_on=['AutopsyID_OFC', 'Hemisphere_OFC'],  
                                         right_on=['AutopsyID_ANG', 'Hemisphere_ANG'], how='inner')

TDP_OFC_ANG = TDP_OFC_ANG.drop(columns=['AutopsyID_ANG', 'Hemisphere_ANG'])
TDP_OFC_ANG = TDP_OFC_ANG.rename(columns={'AutopsyID_OFC': 'AutopsyID',
                                          'Hemisphere_OFC': 'Hemisphere'})

In [38]:
TDP_OFC_ANG

,AutopsyID,BlockID_OFC,Hemisphere,Ferritin_OFC,GFAP_OFC,TDP43_OFC,INDDID_OFC,importID_OFC,0Cont1TDP2Tau_OFC,1ExcludeALSorLATE_OFC,...,BlockID_ANG,Ferritin_ANG,GFAP_ANG,TDP43_ANG,INDDID_ANG,importID_ANG,0Cont1TDP2Tau_ANG,1ExcludeALSorLATE_ANG,1PSP0not_ANG,NPDx1_ANG
0,1995-005,1995-005-04E,R,0.026164,0.051511,0.004535,111517.0,111517.0,1,0,...,1995-005-08E,0.022526,NaN,0.002226,111517.0,111517.0,1,0,0,Frontotemporal lobar degeneration with TDP inc...
1,1995-217,1995-217-05F,R,0.017214,0.037690,0.000888,111863.0,111863.0,1,0,...,1995-217-08F,0.031052,NaN,0.001599,111863.0,111863.0,1,0,0,Frontotemporal lobar degeneration with TDP inc...
2,1999-170,1999-170-03E,L,0.027969,0.036721,0.002450,103640.0,103640.0,1,0,...,1999-170-06E,0.011858,NaN,0.001215,103640.0,103640.0,1,0,0,Frontotemporal lobar degeneration with TDP inc...
3,1999-224,1999-224-31F,L,0.013363,0.042514,0.000669,112298.0,112298.0,1,0,...,1999-224-35F,0.009827,NaN,0.001056,112298.0,112298.0,1,0,0,Frontotemporal lobar degeneration with TDP inc...
4,2000-149,2000-149-40F,R,0.016860,0.034996,0.005378,113227.0,113227.0,1,0,...,2000-149-42F,0.019514,NaN,0.008545,113227.0,113227.0,1,0,0,Frontotemporal lobar degeneration with TDP inc...
5,2004-008,2004-008-05F,L,0.024742,0.047877,0.001934,110338.0,110338.0,1,0,...,2004-008-13F,0.021446,NaN,0.000265,110338.0,110338.0,1,0,0,Frontotemporal lobar degeneration with TDP inc...
6,2004-262,2004-262-29F,L,0.020728,0.032698,0.001649,110361.0,110361.0,1,0,...,2004-262-31F,0.022496,NaN,0.000579,110361.0,110361.0,1,0,0,Frontotemporal lobar degeneration with TDP inc...
7,2007-039,2007-039-11F,L,0.019655,0.056007,0.001083,110581.0,110581.0,1,0,...,2007-039-14F,0.053325,NaN,0.000014,110581.0,110581.0,1,0,0,Frontotemporal lobar degeneration with TDP inc...
8,2008-001,2008-001-24F,R,0.007760,0.030444,0.000063,111092.0,111092.0,1,0,...,2008-001-26F,0.020279,NaN,0.000002,111092.0,111092.0,1,0,0,Frontotemporal lobar degeneration with TDP inc...
9,2008-147,2008-147-06E,L,0.020561,0.040145,0.003692,103601.0,103601.0,1,0,...,2008-147-36F,0.024599,NaN,0.002295,103601.0,103601.0,1,0,0,Frontotemporal lobar degeneration with TDP inc...


In [39]:
# Merge M1, V1 based on AutopsyID & Hemisphere
TDP_M1_V1 = pd.merge(tdp_M1, tdp_V1, left_on=['AutopsyID_M1', 'Hemisphere_M1'],  
                                         right_on=['AutopsyID_V1', 'Hemisphere_V1'], how='inner')

TDP_M1_V1 = TDP_M1_V1.drop(columns=['AutopsyID_V1', 'Hemisphere_V1'])
TDP_M1_V1 = TDP_M1_V1.rename(columns={'AutopsyID_M1': 'AutopsyID',
                                          'Hemisphere_M1': 'Hemisphere'})

In [40]:
TDP_M1_V1_PSP1 = TDP_M1_V1[TDP_M1_V1['1PSP0not_V1'] == 1]

In [41]:
# Merge MFC, V1 based on AutopsyID & Hemisphere
TDP_MFC_V1 = pd.merge(tdp_MFC, tdp_V1, left_on=['AutopsyID_MFC', 'Hemisphere_MFC'],  
                                         right_on=['AutopsyID_V1', 'Hemisphere_V1'], how='inner')

TDP_MFC_V1 = TDP_MFC_V1.drop(columns=['AutopsyID_V1', 'Hemisphere_V1'])
TDP_MFC_V1 = TDP_MFC_V1.rename(columns={'AutopsyID_MFC': 'AutopsyID',
                                          'Hemisphere_MFC': 'Hemisphere'})

In [42]:
TDP_MFC_V1_PSP1 = TDP_MFC_V1[TDP_MFC_V1['1PSP0not_V1'] == 1]

In [43]:
# Merge OFC, V1 based on AutopsyID & Hemisphere
TDP_OFC_V1 = pd.merge(tdp_OFC, tdp_V1, left_on=['AutopsyID_OFC', 'Hemisphere_OFC'],  
                                         right_on=['AutopsyID_V1', 'Hemisphere_V1'], how='inner')

TDP_OFC_V1 = TDP_OFC_V1.drop(columns=['AutopsyID_V1', 'Hemisphere_V1'])
TDP_OFC_V1 = TDP_OFC_V1.rename(columns={'AutopsyID_OFC': 'AutopsyID',
                                          'Hemisphere_OFC': 'Hemisphere'})

In [44]:
TDP_OFC_V1_PSP1 = TDP_OFC_V1[TDP_OFC_V1['1PSP0not_V1'] == 1]

# Mediation Analysis

## AT8_MFC $\rightarrow$ Ferritin_MFC $\rightarrow$ AT8_ANG

In [45]:
# Model for the mediator
mediator_model = sm.OLS(TAU_MFC_ANG['Ferritin_MFC'], sm.add_constant(TAU_MFC_ANG['AT8_MFC']))

# Model for the dependent variable
outcome_model = sm.OLS(TAU_MFC_ANG['AT8_ANG'], sm.add_constant(TAU_MFC_ANG[['AT8_MFC', 'Ferritin_MFC']]))

# Create the mediation analysis model / AT8_MFC: exposure = Independent Variable, Ferritin_MFC: Mediator
med_analysis = Mediation(outcome_model, mediator_model, 'AT8_MFC', 'Ferritin_MFC')

# Fit the model with bootstrapping
boot_results = med_analysis.fit(n_rep=1000)  # Using 1000 bootstrap samples

boot_results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),0.010799,-0.024658,0.066515,0.632
ACME (treated),0.010799,-0.024658,0.066515,0.632
ADE (control),0.718374,0.569336,0.855772,0.000
ADE (treated),0.718374,0.569336,0.855772,0.000
Total effect,0.729173,0.579203,0.865907,0.000
Prop. mediated (control),0.007728,-0.033197,0.085947,0.632
Prop. mediated (treated),0.007728,-0.033197,0.085947,0.632
ACME (average),0.010799,-0.024658,0.066515,0.632
ADE (average),0.718374,0.569336,0.855772,0.000
Prop. mediated (average),0.007728,-0.033197,0.085947,0.632


In [46]:
from statsmodels.formula.api import ols
# Step 1: Regress the DV on the IV
model_1 = ols('AT8_ANG ~ AT8_MFC', data=TAU_MFC_ANG).fit()
a = model_1.params['AT8_MFC']  # Use the name of the IV here, not the DV

# Step 2: Regress the MV on the IV
model_2 = ols('Ferritin_MFC ~ AT8_MFC', data=TAU_MFC_ANG).fit()
b = model_2.params['AT8_MFC']  # Again, use the IV's name

# Step 3: Regress the DV on both the IV and MV
model_3 = ols('AT8_ANG ~ AT8_MFC + Ferritin_MFC', data=TAU_MFC_ANG).fit()
c_prime = model_3.params['AT8_MFC']  # And again, use the IV's name

# The parameter for the mediator in the third model would be:
b_prime = model_3.params['Ferritin_MFC']

In [47]:
# print(a) # b1: Y = b0 + b1*X
# print(b) # b2: M = b0 + b2*M
# print(c_prime) #b4: Y = b0 + b4*X + b3*M
# print(b_prime) #b3

# ACME
print(f'ACME: {a - c_prime} or {b * b_prime}')
# ADE
print(f'ADE: {c_prime}')
# Total Effect
print(f'Total Effect: {a}')


ACME: 0.010357368368158482 or 0.010357368368158537
ADE: 0.7231818127857351
Total Effect: 0.7335391811538936


## AT8_OFC $\rightarrow$ Ferritin_OFC $\rightarrow$ AT8_ANG

In [48]:
# Model for the mediator
mediator_model = sm.OLS(TAU_OFC_ANG['Ferritin_OFC'], sm.add_constant(TAU_OFC_ANG['AT8_OFC']))

# Model for the dependent variable
outcome_model = sm.OLS(TAU_OFC_ANG['AT8_ANG'], sm.add_constant(TAU_OFC_ANG[['AT8_OFC', 'Ferritin_OFC']]))

# Create the mediation analysis model
med_analysis = Mediation(outcome_model, mediator_model, 'AT8_OFC', 'Ferritin_OFC')

# Fit the model with bootstrapping
boot_results = med_analysis.fit(n_rep=1000)  # Using 1000 bootstrap samples

boot_results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),0.012366,-0.052836,0.102250,0.742
ACME (treated),0.012366,-0.052836,0.102250,0.742
ADE (control),0.527976,0.300475,0.768601,0.000
ADE (treated),0.527976,0.300475,0.768601,0.000
Total effect,0.540342,0.307267,0.774629,0.000
Prop. mediated (control),0.010427,-0.123056,0.173121,0.742
Prop. mediated (treated),0.010427,-0.123056,0.173121,0.742
ACME (average),0.012366,-0.052836,0.102250,0.742
ADE (average),0.527976,0.300475,0.768601,0.000
Prop. mediated (average),0.010427,-0.123056,0.173121,0.742


## TDP43_MFC $\rightarrow$ Ferritin_MFC $\rightarrow$ TDP43_ANG

In [49]:
# Define the mediation model
# Model for the mediator
mediator_model = sm.OLS(TDP_MFC_ANG['Ferritin_MFC'], sm.add_constant(TDP_MFC_ANG['TDP43_MFC']))

# Model for the dependent variable
outcome_model = sm.OLS(TDP_MFC_ANG['TDP43_ANG'], sm.add_constant(TDP_MFC_ANG[['TDP43_MFC', 'Ferritin_MFC']]))

# Create the mediation analysis model
med_analysis = Mediation(outcome_model, mediator_model, 'TDP43_MFC', 'Ferritin_MFC')

# Fit the model with bootstrapping
boot_results = med_analysis.fit(n_rep=1000)  # Using 1000 bootstrap samples

boot_results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),-0.140307,-1.220538,0.637926,0.766
ACME (treated),-0.140307,-1.220538,0.637926,0.766
ADE (control),5.043954,1.546105,8.279787,0.002
ADE (treated),5.043954,1.546105,8.279787,0.002
Total effect,4.903647,1.227779,8.193417,0.004
Prop. mediated (control),-0.011020,-0.393423,0.171845,0.770
Prop. mediated (treated),-0.011020,-0.393423,0.171845,0.770
ACME (average),-0.140307,-1.220538,0.637926,0.766
ADE (average),5.043954,1.546105,8.279787,0.002
Prop. mediated (average),-0.011020,-0.393423,0.171845,0.770


## TDP43_OFC $\rightarrow$ Ferritin_OFC $\rightarrow$ TDP43_ANG

In [50]:
# Define the mediation model
# Model for the mediator
mediator_model = sm.OLS(TDP_OFC_ANG['Ferritin_OFC'], sm.add_constant(TDP_OFC_ANG['TDP43_OFC']))

# Model for the dependent variable
outcome_model = sm.OLS(TDP_OFC_ANG['TDP43_ANG'], sm.add_constant(TDP_OFC_ANG[['TDP43_OFC', 'Ferritin_OFC']]))

# Create the mediation analysis model
med_analysis = Mediation(outcome_model, mediator_model, 'TDP43_OFC', 'Ferritin_OFC')

# Fit the model with bootstrapping
boot_results = med_analysis.fit(n_rep=1000)  # Using 1000 bootstrap samples

boot_results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),-0.011561,-0.131355,0.093643,0.826
ACME (treated),-0.011561,-0.131355,0.093643,0.826
ADE (control),0.598638,0.234024,1.002542,0.004
ADE (treated),0.598638,0.234024,1.002542,0.004
Total effect,0.587077,0.210127,0.969825,0.006
Prop. mediated (control),-0.009280,-0.296621,0.168296,0.824
Prop. mediated (treated),-0.009280,-0.296621,0.168296,0.824
ACME (average),-0.011561,-0.131355,0.093643,0.826
ADE (average),0.598638,0.234024,1.002542,0.004
Prop. mediated (average),-0.009280,-0.296621,0.168296,0.824


In [51]:
# Try the average Ferritin avg between MFC & ANG
# OFC & ANG

## AT8_MFC $\rightarrow$ Ferritin_MFC+ANG $\rightarrow$ AT8_ANG

In [52]:
TAU_MFC_ANG['Ferritin_MFC_ANG'] = TAU_MFC_ANG[['Ferritin_MFC', 'Ferritin_ANG']].mean(axis=1)

In [53]:
# Model for the mediator
mediator_model = sm.OLS(TAU_MFC_ANG['Ferritin_MFC_ANG'], sm.add_constant(TAU_MFC_ANG['AT8_MFC']))

# Model for the dependent variable
outcome_model = sm.OLS(TAU_MFC_ANG['AT8_ANG'], sm.add_constant(TAU_MFC_ANG[['AT8_MFC', 'Ferritin_MFC_ANG']]))

# Create the mediation analysis model / AT8_MFC: exposure = Independent Variable, Ferritin_MFC: Mediator
med_analysis = Mediation(outcome_model, mediator_model, 'AT8_MFC', 'Ferritin_MFC_ANG')

# Fit the model with bootstrapping
boot_results = med_analysis.fit(n_rep=1000)  # Using 1000 bootstrap samples

boot_results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),-0.001495,-0.032806,0.027108,0.916
ACME (treated),-0.001495,-0.032806,0.027108,0.916
ADE (control),0.736284,0.602000,0.883467,0.000
ADE (treated),0.736284,0.602000,0.883467,0.000
Total effect,0.734789,0.596660,0.887268,0.000
Prop. mediated (control),-0.000427,-0.045652,0.037434,0.916
Prop. mediated (treated),-0.000427,-0.045652,0.037434,0.916
ACME (average),-0.001495,-0.032806,0.027108,0.916
ADE (average),0.736284,0.602000,0.883467,0.000
Prop. mediated (average),-0.000427,-0.045652,0.037434,0.916


## AT8_OFC $\rightarrow$ Ferritin_OFC+ANG $\rightarrow$ AT8_ANG

In [54]:
TAU_OFC_ANG['Ferritin_OFC_ANG'] = TAU_OFC_ANG[['Ferritin_OFC', 'Ferritin_ANG']].mean(axis=1)

In [55]:
# Model for the mediator
mediator_model = sm.OLS(TAU_OFC_ANG['Ferritin_OFC_ANG'], sm.add_constant(TAU_OFC_ANG['AT8_OFC']))

# Model for the dependent variable
outcome_model = sm.OLS(TAU_OFC_ANG['AT8_ANG'], sm.add_constant(TAU_OFC_ANG[['AT8_OFC', 'Ferritin_OFC_ANG']]))

# Create the mediation analysis model
med_analysis = Mediation(outcome_model, mediator_model, 'AT8_OFC', 'Ferritin_OFC_ANG')

# Fit the model with bootstrapping
boot_results = med_analysis.fit(n_rep=1000)  # Using 1000 bootstrap samples

boot_results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),0.002345,-0.048104,0.060183,0.912
ACME (treated),0.002345,-0.048104,0.060183,0.912
ADE (control),0.542891,0.312962,0.764194,0.000
ADE (treated),0.542891,0.312962,0.764194,0.000
Total effect,0.545236,0.313217,0.766367,0.000
Prop. mediated (control),0.001352,-0.106776,0.112751,0.912
Prop. mediated (treated),0.001352,-0.106776,0.112751,0.912
ACME (average),0.002345,-0.048104,0.060183,0.912
ADE (average),0.542891,0.312962,0.764194,0.000
Prop. mediated (average),0.001352,-0.106776,0.112751,0.912


## TDP43_MFC $\rightarrow$ Ferritin_MFC+ANG $\rightarrow$ TDP43_ANG

In [56]:
TDP_MFC_ANG['Ferritin_MFC_ANG'] = TDP_MFC_ANG[['Ferritin_MFC', 'Ferritin_ANG']].mean(axis=1)

In [57]:
# Define the mediation model
# Model for the mediator
mediator_model = sm.OLS(TDP_MFC_ANG['Ferritin_MFC_ANG'], sm.add_constant(TDP_MFC_ANG['TDP43_MFC']))

# Model for the dependent variable
outcome_model = sm.OLS(TDP_MFC_ANG['TDP43_ANG'], sm.add_constant(TDP_MFC_ANG[['TDP43_MFC', 'Ferritin_MFC_ANG']]))

# Create the mediation analysis model
med_analysis = Mediation(outcome_model, mediator_model, 'TDP43_MFC', 'Ferritin_MFC_ANG')

# Fit the model with bootstrapping
boot_results = med_analysis.fit(n_rep=1000)  # Using 1000 bootstrap samples

boot_results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),0.016103,-0.944536,0.975841,0.956
ACME (treated),0.016103,-0.944536,0.975841,0.956
ADE (control),4.899011,1.610891,8.324612,0.000
ADE (treated),4.899011,1.610891,8.324612,0.000
Total effect,4.915114,1.614572,8.369770,0.002
Prop. mediated (control),0.000996,-0.366645,0.243274,0.954
Prop. mediated (treated),0.000996,-0.366645,0.243274,0.954
ACME (average),0.016103,-0.944536,0.975841,0.956
ADE (average),4.899011,1.610891,8.324612,0.000
Prop. mediated (average),0.000996,-0.366645,0.243274,0.954


## TDP43_OFC $\rightarrow$ Ferritin_OFC+ANG $\rightarrow$ TDP43_ANG

In [58]:
TDP_OFC_ANG['Ferritin_OFC_ANG'] = TDP_OFC_ANG[['Ferritin_OFC', 'Ferritin_ANG']].mean(axis=1)

In [59]:
# Define the mediation model
# Model for the mediator
mediator_model = sm.OLS(TDP_OFC_ANG['Ferritin_OFC_ANG'], sm.add_constant(TDP_OFC_ANG['TDP43_OFC']))

# Model for the dependent variable
outcome_model = sm.OLS(TDP_OFC_ANG['TDP43_ANG'], sm.add_constant(TDP_OFC_ANG[['TDP43_OFC', 'Ferritin_OFC_ANG']]))

# Create the mediation analysis model
med_analysis = Mediation(outcome_model, mediator_model, 'TDP43_OFC', 'Ferritin_OFC_ANG')

# Fit the model with bootstrapping
boot_results = med_analysis.fit(n_rep=1000)  # Using 1000 bootstrap samples

boot_results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),-0.002494,-0.106886,0.101139,0.942
ACME (treated),-0.002494,-0.106886,0.101139,0.942
ADE (control),0.572925,0.195554,0.948519,0.004
ADE (treated),0.572925,0.195554,0.948519,0.004
Total effect,0.570431,0.177553,0.941952,0.006
Prop. mediated (control),-0.001046,-0.286219,0.190540,0.944
Prop. mediated (treated),-0.001046,-0.286219,0.190540,0.944
ACME (average),-0.002494,-0.106886,0.101139,0.942
ADE (average),0.572925,0.195554,0.948519,0.004
Prop. mediated (average),-0.001046,-0.286219,0.190540,0.944


# 01/23/2024

## TAU

### TAU(M1) $\rightarrow$ Ferritin(M1) $\rightarrow$ TAU(V1)

In [60]:
data = TAU_M1_V1
ind_var = 'AT8_M1'
mediator = 'Ferritin_M1'
dep_var = 'AT8_V1'

# Model for the mediator
mediator_model = sm.OLS(data[mediator], sm.add_constant(data[ind_var]))

# Model for the dependent variable
outcome_model = sm.OLS(data[dep_var], sm.add_constant(data[[ind_var, mediator]]))

# Create the mediation analysis model / AT8_MFC: exposure = Independent Variable, Ferritin_MFC: Mediator
med_analysis = Mediation(outcome_model, mediator_model, ind_var, mediator)

# Fit the model with bootstrapping
boot_results = med_analysis.fit(n_rep=1000)  # Using 1000 bootstrap samples

boot_results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),0.000222,-0.000824,0.001715,0.682
ACME (treated),0.000222,-0.000824,0.001715,0.682
ADE (control),-0.001107,-0.003636,0.001452,0.398
ADE (treated),-0.001107,-0.003636,0.001452,0.398
Total effect,-0.000885,-0.003541,0.001736,0.524
Prop. mediated (control),-0.013689,-7.359664,4.155528,0.922
Prop. mediated (treated),-0.013689,-7.359664,4.155528,0.922
ACME (average),0.000222,-0.000824,0.001715,0.682
ADE (average),-0.001107,-0.003636,0.001452,0.398
Prop. mediated (average),-0.013689,-7.359664,4.155528,0.922


In [61]:
data = TAU_M1_V1_PSP1
ind_var = 'AT8_M1'
mediator = 'Ferritin_M1'
dep_var = 'AT8_V1'

# Model for the mediator
mediator_model = sm.OLS(data[mediator], sm.add_constant(data[ind_var]))

# Model for the dependent variable
outcome_model = sm.OLS(data[dep_var], sm.add_constant(data[[ind_var, mediator]]))

# Create the mediation analysis model / AT8_MFC: exposure = Independent Variable, Ferritin_MFC: Mediator
med_analysis = Mediation(outcome_model, mediator_model, ind_var, mediator)

# Fit the model with bootstrapping
boot_results = med_analysis.fit(n_rep=1000)  # Using 1000 bootstrap samples

boot_results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),-0.000723,-0.011563,0.009389,0.872
ACME (treated),-0.000723,-0.011563,0.009389,0.872
ADE (control),-0.002751,-0.017074,0.011538,0.698
ADE (treated),-0.002751,-0.017074,0.011538,0.698
Total effect,-0.003474,-0.018380,0.010774,0.622
Prop. mediated (control),0.085327,-4.540376,6.942342,0.846
Prop. mediated (treated),0.085327,-4.540376,6.942342,0.846
ACME (average),-0.000723,-0.011563,0.009389,0.872
ADE (average),-0.002751,-0.017074,0.011538,0.698
Prop. mediated (average),0.085327,-4.540376,6.942342,0.846


### TAU(M1) $\rightarrow$ Ferritin(M1+V1) $\rightarrow$ TAU(V1)

In [62]:
TAU_M1_V1['Ferritin_M1_V1'] = TAU_M1_V1[['Ferritin_M1', 'Ferritin_V1']].mean(axis=1)

In [63]:
data = TAU_M1_V1
ind_var = 'AT8_M1'
mediator = 'Ferritin_M1_V1'
dep_var = 'AT8_V1'

# Model for the mediator
mediator_model = sm.OLS(data[mediator], sm.add_constant(data[ind_var]))

# Model for the dependent variable
outcome_model = sm.OLS(data[dep_var], sm.add_constant(data[[ind_var, mediator]]))

# Create the mediation analysis model / AT8_MFC: exposure = Independent Variable, Ferritin_MFC: Mediator
med_analysis = Mediation(outcome_model, mediator_model, ind_var, mediator)

# Fit the model with bootstrapping
boot_results = med_analysis.fit(n_rep=1000)  # Using 1000 bootstrap samples

boot_results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),0.000152,-0.001132,0.001731,0.808
ACME (treated),0.000152,-0.001132,0.001731,0.808
ADE (control),-0.001155,-0.004022,0.001502,0.426
ADE (treated),-0.001155,-0.004022,0.001502,0.426
Total effect,-0.001003,-0.003850,0.001705,0.468
Prop. mediated (control),-0.013704,-6.607892,4.908464,0.936
Prop. mediated (treated),-0.013704,-6.607892,4.908464,0.936
ACME (average),0.000152,-0.001132,0.001731,0.808
ADE (average),-0.001155,-0.004022,0.001502,0.426
Prop. mediated (average),-0.013704,-6.607892,4.908464,0.936


In [64]:
TAU_M1_V1_PSP1['Ferritin_M1_V1'] = TAU_M1_V1_PSP1[['Ferritin_M1', 'Ferritin_V1']].mean(axis=1)

/var/folders/d_/hlp96bh14c98xy43dzm6gp6c0000gn/T/ipykernel_47812/2801127996.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TAU_M1_V1_PSP1['Ferritin_M1_V1'] = TAU_M1_V1_PSP1[['Ferritin_M1', 'Ferritin_V1']].mean(axis=1)


In [65]:
data = TAU_M1_V1_PSP1
ind_var = 'AT8_M1'
mediator = 'Ferritin_M1_V1'
dep_var = 'AT8_V1'

# Model for the mediator
mediator_model = sm.OLS(data[mediator], sm.add_constant(data[ind_var]))

# Model for the dependent variable
outcome_model = sm.OLS(data[dep_var], sm.add_constant(data[[ind_var, mediator]]))

# Create the mediation analysis model / AT8_MFC: exposure = Independent Variable, Ferritin_MFC: Mediator
med_analysis = Mediation(outcome_model, mediator_model, ind_var, mediator)

# Fit the model with bootstrapping
boot_results = med_analysis.fit(n_rep=1000)  # Using 1000 bootstrap samples

boot_results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),-0.000017,-0.006977,0.006871,0.944
ACME (treated),-0.000017,-0.006977,0.006871,0.944
ADE (control),-0.003484,-0.016399,0.008970,0.594
ADE (treated),-0.003484,-0.016399,0.008970,0.594
Total effect,-0.003501,-0.017693,0.010257,0.638
Prop. mediated (control),0.060339,-2.937198,3.260264,0.770
Prop. mediated (treated),0.060339,-2.937198,3.260264,0.770
ACME (average),-0.000017,-0.006977,0.006871,0.944
ADE (average),-0.003484,-0.016399,0.008970,0.594
Prop. mediated (average),0.060339,-2.937198,3.260264,0.770


### TAU(MFC) $\rightarrow$ Ferritin(MFC) $\rightarrow$ TAU(V1)

In [66]:
data = TAU_MFC_V1
ind_var = 'AT8_MFC'
mediator = 'Ferritin_MFC'
dep_var = 'AT8_V1'

# Model for the mediator
mediator_model = sm.OLS(data[mediator], sm.add_constant(data[ind_var]))

# Model for the dependent variable
outcome_model = sm.OLS(data[dep_var], sm.add_constant(data[[ind_var, mediator]]))

# Create the mediation analysis model / AT8_MFC: exposure = Independent Variable, Ferritin_MFC: Mediator
med_analysis = Mediation(outcome_model, mediator_model, ind_var, mediator)

# Fit the model with bootstrapping
boot_results = med_analysis.fit(n_rep=1000)  # Using 1000 bootstrap samples

boot_results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),-0.000073,-0.000412,0.000150,0.578
ACME (treated),-0.000073,-0.000412,0.000150,0.578
ADE (control),0.000596,0.000056,0.001196,0.036
ADE (treated),0.000596,0.000056,0.001196,0.036
Total effect,0.000524,-0.000074,0.001080,0.084
Prop. mediated (control),-0.072249,-1.566183,1.609290,0.630
Prop. mediated (treated),-0.072249,-1.566183,1.609290,0.630
ACME (average),-0.000073,-0.000412,0.000150,0.578
ADE (average),0.000596,0.000056,0.001196,0.036
Prop. mediated (average),-0.072249,-1.566183,1.609290,0.630


In [67]:
data = TAU_MFC_V1_PSP1
ind_var = 'AT8_MFC'
mediator = 'Ferritin_MFC'
dep_var = 'AT8_V1'

# Model for the mediator
mediator_model = sm.OLS(data[mediator], sm.add_constant(data[ind_var]))

# Model for the dependent variable
outcome_model = sm.OLS(data[dep_var], sm.add_constant(data[[ind_var, mediator]]))

# Create the mediation analysis model / AT8_MFC: exposure = Independent Variable, Ferritin_MFC: Mediator
med_analysis = Mediation(outcome_model, mediator_model, ind_var, mediator)

# Fit the model with bootstrapping
boot_results = med_analysis.fit(n_rep=1000)  # Using 1000 bootstrap samples

boot_results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),0.002962,-0.026847,0.033263,0.818
ACME (treated),0.002962,-0.026847,0.033263,0.818
ADE (control),-0.008590,-0.053475,0.035481,0.716
ADE (treated),-0.008590,-0.053475,0.035481,0.716
Total effect,-0.005628,-0.044909,0.034544,0.808
Prop. mediated (control),0.018718,-5.140058,7.227917,0.974
Prop. mediated (treated),0.018718,-5.140058,7.227917,0.974
ACME (average),0.002962,-0.026847,0.033263,0.818
ADE (average),-0.008590,-0.053475,0.035481,0.716
Prop. mediated (average),0.018718,-5.140058,7.227917,0.974


### TAU(MFC) $\rightarrow$ Ferritin(MFC+V1) $\rightarrow$ TAU(V1)

In [68]:
TAU_MFC_V1['Ferritin_MFC_V1'] = TAU_MFC_V1[['Ferritin_MFC', 'Ferritin_V1']].mean(axis=1)

In [69]:
data = TAU_MFC_V1
ind_var = 'AT8_MFC'
mediator = 'Ferritin_MFC_V1'
dep_var = 'AT8_V1'

# Model for the mediator
mediator_model = sm.OLS(data[mediator], sm.add_constant(data[ind_var]))

# Model for the dependent variable
outcome_model = sm.OLS(data[dep_var], sm.add_constant(data[[ind_var, mediator]]))

# Create the mediation analysis model / AT8_MFC: exposure = Independent Variable, Ferritin_MFC: Mediator
med_analysis = Mediation(outcome_model, mediator_model, ind_var, mediator)

# Fit the model with bootstrapping
boot_results = med_analysis.fit(n_rep=1000)  # Using 1000 bootstrap samples

boot_results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),-0.000056,-0.000440,0.000289,0.740
ACME (treated),-0.000056,-0.000440,0.000289,0.740
ADE (control),0.000574,-0.000103,0.001209,0.082
ADE (treated),0.000574,-0.000103,0.001209,0.082
Total effect,0.000518,-0.000056,0.001097,0.072
Prop. mediated (control),-0.072761,-2.955972,1.952720,0.752
Prop. mediated (treated),-0.072761,-2.955972,1.952720,0.752
ACME (average),-0.000056,-0.000440,0.000289,0.740
ADE (average),0.000574,-0.000103,0.001209,0.082
Prop. mediated (average),-0.072761,-2.955972,1.952720,0.752


In [70]:
TAU_MFC_V1_PSP1['Ferritin_MFC_V1'] = TAU_MFC_V1_PSP1[['Ferritin_MFC', 'Ferritin_V1']].mean(axis=1)

/var/folders/d_/hlp96bh14c98xy43dzm6gp6c0000gn/T/ipykernel_47812/3735018093.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TAU_MFC_V1_PSP1['Ferritin_MFC_V1'] = TAU_MFC_V1_PSP1[['Ferritin_MFC', 'Ferritin_V1']].mean(axis=1)


In [71]:
data = TAU_MFC_V1_PSP1
ind_var = 'AT8_MFC'
mediator = 'Ferritin_MFC_V1'
dep_var = 'AT8_V1'

# Model for the mediator
mediator_model = sm.OLS(data[mediator], sm.add_constant(data[ind_var]))

# Model for the dependent variable
outcome_model = sm.OLS(data[dep_var], sm.add_constant(data[[ind_var, mediator]]))

# Create the mediation analysis model / AT8_MFC: exposure = Independent Variable, Ferritin_MFC: Mediator
med_analysis = Mediation(outcome_model, mediator_model, ind_var, mediator)

# Fit the model with bootstrapping
boot_results = med_analysis.fit(n_rep=1000)  # Using 1000 bootstrap samples

boot_results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),0.001935,-0.015231,0.022415,0.832
ACME (treated),0.001935,-0.015231,0.022415,0.832
ADE (control),-0.006912,-0.044940,0.031849,0.728
ADE (treated),-0.006912,-0.044940,0.031849,0.728
Total effect,-0.004978,-0.044391,0.035030,0.806
Prop. mediated (control),0.014414,-4.293277,4.096144,0.894
Prop. mediated (treated),0.014414,-4.293277,4.096144,0.894
ACME (average),0.001935,-0.015231,0.022415,0.832
ADE (average),-0.006912,-0.044940,0.031849,0.728
Prop. mediated (average),0.014414,-4.293277,4.096144,0.894


### TAU(OFC) $\rightarrow$ Ferritin(OFC) $\rightarrow$ TAU(V1)

In [72]:
data = TAU_OFC_V1
ind_var = 'AT8_OFC'
mediator = 'Ferritin_OFC'
dep_var = 'AT8_V1'

# Model for the mediator
mediator_model = sm.OLS(data[mediator], sm.add_constant(data[ind_var]))

# Model for the dependent variable
outcome_model = sm.OLS(data[dep_var], sm.add_constant(data[[ind_var, mediator]]))

# Create the mediation analysis model / AT8_MFC: exposure = Independent Variable, Ferritin_MFC: Mediator
med_analysis = Mediation(outcome_model, mediator_model, ind_var, mediator)

# Fit the model with bootstrapping
boot_results = med_analysis.fit(n_rep=1000)  # Using 1000 bootstrap samples

boot_results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),-0.000053,-0.000412,0.000249,0.718
ACME (treated),-0.000053,-0.000412,0.000249,0.718
ADE (control),0.000420,-0.000244,0.001124,0.264
ADE (treated),0.000420,-0.000244,0.001124,0.264
Total effect,0.000367,-0.000321,0.001082,0.336
Prop. mediated (control),-0.024123,-2.841748,2.954714,0.862
Prop. mediated (treated),-0.024123,-2.841748,2.954714,0.862
ACME (average),-0.000053,-0.000412,0.000249,0.718
ADE (average),0.000420,-0.000244,0.001124,0.264
Prop. mediated (average),-0.024123,-2.841748,2.954714,0.862


In [73]:
data = TAU_OFC_V1_PSP1
ind_var = 'AT8_OFC'
mediator = 'Ferritin_OFC'
dep_var = 'AT8_V1'

# Model for the mediator
mediator_model = sm.OLS(data[mediator], sm.add_constant(data[ind_var]))

# Model for the dependent variable
outcome_model = sm.OLS(data[dep_var], sm.add_constant(data[[ind_var, mediator]]))

# Create the mediation analysis model / AT8_MFC: exposure = Independent Variable, Ferritin_MFC: Mediator
med_analysis = Mediation(outcome_model, mediator_model, ind_var, mediator)

# Fit the model with bootstrapping
boot_results = med_analysis.fit(n_rep=1000)  # Using 1000 bootstrap samples

boot_results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),0.073758,-0.039079,0.253701,0.228
ACME (treated),0.073758,-0.039079,0.253701,0.228
ADE (control),-0.049497,-0.227139,0.122940,0.580
ADE (treated),-0.049497,-0.227139,0.122940,0.580
Total effect,0.024261,-0.171630,0.235612,0.812
Prop. mediated (control),0.371894,-11.148655,11.945350,0.752
Prop. mediated (treated),0.371894,-11.148655,11.945350,0.752
ACME (average),0.073758,-0.039079,0.253701,0.228
ADE (average),-0.049497,-0.227139,0.122940,0.580
Prop. mediated (average),0.371894,-11.148655,11.945350,0.752


### TAU(OFC) $\rightarrow$ Ferritin(OFC+V1) $\rightarrow$ TAU(V1)

In [74]:
TAU_OFC_V1['Ferritin_OFC_V1'] = TAU_OFC_V1[['Ferritin_OFC', 'Ferritin_V1']].mean(axis=1)

In [75]:
data = TAU_OFC_V1
ind_var = 'AT8_OFC'
mediator = 'Ferritin_OFC_V1'
dep_var = 'AT8_V1'

# Model for the mediator
mediator_model = sm.OLS(data[mediator], sm.add_constant(data[ind_var]))

# Model for the dependent variable
outcome_model = sm.OLS(data[dep_var], sm.add_constant(data[[ind_var, mediator]]))

# Create the mediation analysis model / AT8_MFC: exposure = Independent Variable, Ferritin_MFC: Mediator
med_analysis = Mediation(outcome_model, mediator_model, ind_var, mediator)

# Fit the model with bootstrapping
boot_results = med_analysis.fit(n_rep=1000)  # Using 1000 bootstrap samples

boot_results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),-0.000012,-0.000364,0.000296,0.962
ACME (treated),-0.000012,-0.000364,0.000296,0.962
ADE (control),0.000360,-0.000361,0.001078,0.336
ADE (treated),0.000360,-0.000361,0.001078,0.336
Total effect,0.000348,-0.000368,0.001052,0.332
Prop. mediated (control),-0.000818,-2.810515,2.085810,0.990
Prop. mediated (treated),-0.000818,-2.810515,2.085810,0.990
ACME (average),-0.000012,-0.000364,0.000296,0.962
ADE (average),0.000360,-0.000361,0.001078,0.336
Prop. mediated (average),-0.000818,-2.810515,2.085810,0.990


In [76]:
TAU_OFC_V1_PSP1['Ferritin_OFC_V1'] = TAU_OFC_V1_PSP1[['Ferritin_OFC', 'Ferritin_V1']].mean(axis=1)

/var/folders/d_/hlp96bh14c98xy43dzm6gp6c0000gn/T/ipykernel_47812/3189640414.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TAU_OFC_V1_PSP1['Ferritin_OFC_V1'] = TAU_OFC_V1_PSP1[['Ferritin_OFC', 'Ferritin_V1']].mean(axis=1)


In [77]:
data = TAU_OFC_V1_PSP1
ind_var = 'AT8_OFC'
mediator = 'Ferritin_OFC_V1'
dep_var = 'AT8_V1'

# Model for the mediator
mediator_model = sm.OLS(data[mediator], sm.add_constant(data[ind_var]))

# Model for the dependent variable
outcome_model = sm.OLS(data[dep_var], sm.add_constant(data[[ind_var, mediator]]))

# Create the mediation analysis model / AT8_MFC: exposure = Independent Variable, Ferritin_MFC: Mediator
med_analysis = Mediation(outcome_model, mediator_model, ind_var, mediator)

# Fit the model with bootstrapping
boot_results = med_analysis.fit(n_rep=1000)  # Using 1000 bootstrap samples

boot_results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),0.043643,-0.057856,0.203232,0.470
ACME (treated),0.043643,-0.057856,0.203232,0.470
ADE (control),-0.022001,-0.214962,0.179590,0.820
ADE (treated),-0.022001,-0.214962,0.179590,0.820
Total effect,0.021642,-0.169590,0.214244,0.816
Prop. mediated (control),0.138830,-6.196687,8.029804,0.802
Prop. mediated (treated),0.138830,-6.196687,8.029804,0.802
ACME (average),0.043643,-0.057856,0.203232,0.470
ADE (average),-0.022001,-0.214962,0.179590,0.820
Prop. mediated (average),0.138830,-6.196687,8.029804,0.802


## TDP43

### TDP43(M1) $\rightarrow$ Ferritin(M1) $\rightarrow$ TDP43(V1)

In [78]:
data = TDP_M1_V1
ind_var = 'TDP43_M1'
mediator = 'Ferritin_M1'
dep_var = 'TDP43_V1'

# Model for the mediator
mediator_model = sm.OLS(data[mediator], sm.add_constant(data[ind_var]))

# Model for the dependent variable
outcome_model = sm.OLS(data[dep_var], sm.add_constant(data[[ind_var, mediator]]))

# Create the mediation analysis model / AT8_MFC: exposure = Independent Variable, Ferritin_MFC: Mediator
med_analysis = Mediation(outcome_model, mediator_model, ind_var, mediator)

# Fit the model with bootstrapping
boot_results = med_analysis.fit(n_rep=1000)  # Using 1000 bootstrap samples

boot_results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),-0.043175,-0.152280,0.021346,0.232
ACME (treated),-0.043175,-0.152280,0.021346,0.232
ADE (control),-0.025228,-0.119614,0.065499,0.564
ADE (treated),-0.025228,-0.119614,0.065499,0.564
Total effect,-0.068403,-0.185040,0.038062,0.208
Prop. mediated (control),0.507113,-2.251846,4.054848,0.300
Prop. mediated (treated),0.507113,-2.251846,4.054848,0.300
ACME (average),-0.043175,-0.152280,0.021346,0.232
ADE (average),-0.025228,-0.119614,0.065499,0.564
Prop. mediated (average),0.507113,-2.251846,4.054848,0.300


### TDP(M1) $\rightarrow$ Ferritin(M1+V1) $\rightarrow$ TDP(V1)

In [79]:
TDP_M1_V1['Ferritin_M1_V1'] = TDP_M1_V1[['Ferritin_M1', 'Ferritin_V1']].mean(axis=1)

In [80]:
data = TDP_M1_V1
ind_var = 'TDP43_M1'
mediator = 'Ferritin_M1_V1'
dep_var = 'TDP43_V1'

# Model for the mediator
mediator_model = sm.OLS(data[mediator], sm.add_constant(data[ind_var]))

# Model for the dependent variable
outcome_model = sm.OLS(data[dep_var], sm.add_constant(data[[ind_var, mediator]]))

# Create the mediation analysis model / AT8_MFC: exposure = Independent Variable, Ferritin_MFC: Mediator
med_analysis = Mediation(outcome_model, mediator_model, ind_var, mediator)

# Fit the model with bootstrapping
boot_results = med_analysis.fit(n_rep=1000)  # Using 1000 bootstrap samples

boot_results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),-0.000026,-0.067119,0.073604,0.994
ACME (treated),-0.000026,-0.067119,0.073604,0.994
ADE (control),-0.064062,-0.186338,0.051826,0.302
ADE (treated),-0.064062,-0.186338,0.051826,0.302
Total effect,-0.064088,-0.185495,0.049923,0.312
Prop. mediated (control),0.010302,-4.712712,3.028682,0.958
Prop. mediated (treated),0.010302,-4.712712,3.028682,0.958
ACME (average),-0.000026,-0.067119,0.073604,0.994
ADE (average),-0.064062,-0.186338,0.051826,0.302
Prop. mediated (average),0.010302,-4.712712,3.028682,0.958


### TDP(MFC) $\rightarrow$ Ferritin(MFC) $\rightarrow$ TDP(V1)

In [81]:
data = TDP_MFC_V1
ind_var = 'TDP43_MFC'
mediator = 'Ferritin_MFC'
dep_var = 'TDP43_V1'

# Model for the mediator
mediator_model = sm.OLS(data[mediator], sm.add_constant(data[ind_var]))

# Model for the dependent variable
outcome_model = sm.OLS(data[dep_var], sm.add_constant(data[[ind_var, mediator]]))

# Create the mediation analysis model / AT8_MFC: exposure = Independent Variable, Ferritin_MFC: Mediator
med_analysis = Mediation(outcome_model, mediator_model, ind_var, mediator)

# Fit the model with bootstrapping
boot_results = med_analysis.fit(n_rep=1000)  # Using 1000 bootstrap samples

boot_results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),-0.006093,-0.044640,0.018617,0.718
ACME (treated),-0.006093,-0.044640,0.018617,0.718
ADE (control),0.052893,-0.011558,0.114347,0.096
ADE (treated),0.052893,-0.011558,0.114347,0.096
Total effect,0.046800,-0.018981,0.109320,0.128
Prop. mediated (control),-0.038198,-3.087829,1.648424,0.794
Prop. mediated (treated),-0.038198,-3.087829,1.648424,0.794
ACME (average),-0.006093,-0.044640,0.018617,0.718
ADE (average),0.052893,-0.011558,0.114347,0.096
Prop. mediated (average),-0.038198,-3.087829,1.648424,0.794


### TDP(MFC) $\rightarrow$ Ferritin(MFC+V1) $\rightarrow$ TDP(V1)

In [82]:
TDP_MFC_V1['Ferritin_MFC_V1'] = TDP_MFC_V1[['Ferritin_MFC', 'Ferritin_V1']].mean(axis=1)

In [83]:
data = TDP_MFC_V1
ind_var = 'TDP43_MFC'
mediator = 'Ferritin_MFC_V1'
dep_var = 'TDP43_V1'

# Model for the mediator
mediator_model = sm.OLS(data[mediator], sm.add_constant(data[ind_var]))

# Model for the dependent variable
outcome_model = sm.OLS(data[dep_var], sm.add_constant(data[[ind_var, mediator]]))

# Create the mediation analysis model / AT8_MFC: exposure = Independent Variable, Ferritin_MFC: Mediator
med_analysis = Mediation(outcome_model, mediator_model, ind_var, mediator)

# Fit the model with bootstrapping
boot_results = med_analysis.fit(n_rep=1000)  # Using 1000 bootstrap samples

boot_results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),-0.012011,-0.056859,0.020812,0.522
ACME (treated),-0.012011,-0.056859,0.020812,0.522
ADE (control),0.058159,-0.007637,0.120103,0.092
ADE (treated),0.058159,-0.007637,0.120103,0.092
Total effect,0.046148,-0.019290,0.103969,0.142
Prop. mediated (control),-0.148893,-3.660910,2.206179,0.596
Prop. mediated (treated),-0.148893,-3.660910,2.206179,0.596
ACME (average),-0.012011,-0.056859,0.020812,0.522
ADE (average),0.058159,-0.007637,0.120103,0.092
Prop. mediated (average),-0.148893,-3.660910,2.206179,0.596


### TDP(OFC) $\rightarrow$ Ferritin(OFC) $\rightarrow$ TDP(V1)

In [84]:
data = TDP_OFC_V1
ind_var = 'TDP43_OFC'
mediator = 'Ferritin_OFC'
dep_var = 'TDP43_V1'

# Model for the mediator
mediator_model = sm.OLS(data[mediator], sm.add_constant(data[ind_var]))

# Model for the dependent variable
outcome_model = sm.OLS(data[dep_var], sm.add_constant(data[[ind_var, mediator]]))

# Create the mediation analysis model / AT8_MFC: exposure = Independent Variable, Ferritin_MFC: Mediator
med_analysis = Mediation(outcome_model, mediator_model, ind_var, mediator)

# Fit the model with bootstrapping
boot_results = med_analysis.fit(n_rep=1000)  # Using 1000 bootstrap samples

boot_results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),-0.008417,-0.050132,0.029725,0.612
ACME (treated),-0.008417,-0.050132,0.029725,0.612
ADE (control),0.016965,-0.025230,0.061281,0.466
ADE (treated),0.016965,-0.025230,0.061281,0.466
Total effect,0.008547,-0.046611,0.065678,0.774
Prop. mediated (control),0.220059,-6.533660,11.339983,0.770
Prop. mediated (treated),0.220059,-6.533660,11.339983,0.770
ACME (average),-0.008417,-0.050132,0.029725,0.612
ADE (average),0.016965,-0.025230,0.061281,0.466
Prop. mediated (average),0.220059,-6.533660,11.339983,0.770


### TDP(OFC) $\rightarrow$ Ferritin(OFC+V1) $\rightarrow$ TDP(V1)

In [85]:
TDP_OFC_V1['Ferritin_OFC_V1'] = TDP_OFC_V1[['Ferritin_OFC', 'Ferritin_V1']].mean(axis=1)

In [86]:
data = TDP_OFC_V1
ind_var = 'TDP43_OFC'
mediator = 'Ferritin_OFC_V1'
dep_var = 'TDP43_V1'

# Model for the mediator
mediator_model = sm.OLS(data[mediator], sm.add_constant(data[ind_var]))

# Model for the dependent variable
outcome_model = sm.OLS(data[dep_var], sm.add_constant(data[[ind_var, mediator]]))

# Create the mediation analysis model / AT8_MFC: exposure = Independent Variable, Ferritin_MFC: Mediator
med_analysis = Mediation(outcome_model, mediator_model, ind_var, mediator)

# Fit the model with bootstrapping
boot_results = med_analysis.fit(n_rep=1000)  # Using 1000 bootstrap samples

boot_results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),-0.015173,-0.068066,0.019736,0.440
ACME (treated),-0.015173,-0.068066,0.019736,0.440
ADE (control),0.023550,-0.037733,0.087444,0.478
ADE (treated),0.023550,-0.037733,0.087444,0.478
Total effect,0.008377,-0.046781,0.064600,0.758
Prop. mediated (control),-0.108127,-9.957315,8.319098,0.866
Prop. mediated (treated),-0.108127,-9.957315,8.319098,0.866
ACME (average),-0.015173,-0.068066,0.019736,0.440
ADE (average),0.023550,-0.037733,0.087444,0.478
Prop. mediated (average),-0.108127,-9.957315,8.319098,0.866
